# Training of base model

In [1]:
import tensorflow as tf
import os
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from utils.helper_utils import HelperUtils
from configuration import Configuration

In [2]:
def prepare_datasets(data_dir):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=Configuration.validation_split,
        subset="training",
        seed=123,
        image_size=(Configuration.img_height, Configuration.img_width),
        batch_size=Configuration.batch_size)

    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=Configuration.validation_split,
        subset="validation",
        seed=123,
        image_size=(Configuration.img_height, Configuration.img_width),
        batch_size=Configuration.batch_size)

    class_names = train_ds.class_names

    num_classes = len(class_names)

    return train_ds, val_ds, num_classes, class_names

Get train and validation datasets

In [3]:
data_dir_name = "simple"
initial_data_dir = os.path.abspath(os.path.join(Configuration.data_location, data_dir_name))
initial_train_ds, initial_val_ds, initial_num_classes, class_names = prepare_datasets(initial_data_dir)

Found 76536 files belonging to 432 classes.
Using 61229 files for training.
Found 76536 files belonging to 432 classes.
Using 15307 files for validation.


Create model

In [4]:
model = Sequential([
    layers.Rescaling(1. / 255, input_shape=(Configuration.img_height, Configuration.img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(initial_num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 90, 90, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 45, 64)        1

Train model

In [5]:
model.fit(
    initial_train_ds,
    validation_data=initial_val_ds,
    epochs=50
)

Epoch 1/50
1914/1914 [==============================] - 78s 38ms/step - loss: 3.1457 - accuracy: 0.3331 - val_loss: 1.5885 - val_accuracy: 0.5876
Epoch 2/50
1914/1914 [==============================] - 73s 38ms/step - loss: 1.0371 - accuracy: 0.7193 - val_loss: 1.1249 - val_accuracy: 0.7079
Epoch 3/50
1914/1914 [==============================] - 72s 38ms/step - loss: 0.4946 - accuracy: 0.8575 - val_loss: 0.8082 - val_accuracy: 0.7930
Epoch 4/50
1914/1914 [==============================] - 73s 38ms/step - loss: 0.2947 - accuracy: 0.9104 - val_loss: 0.8334 - val_accuracy: 0.8074
Epoch 5/50
1914/1914 [==============================] - 71s 37ms/step - loss: 0.2160 - accuracy: 0.9340 - val_loss: 1.1071 - val_accuracy: 0.7617
Epoch 6/50
1914/1914 [==============================] - 69s 36ms/step - loss: 0.1662 - accuracy: 0.9479 - val_loss: 0.8413 - val_accuracy: 0.8422
Epoch 7/50
1914/1914 [==============================] - 69s 36ms/step - loss: 0.1450 - accuracy: 0.9556 - val_loss: 0.7715 -

Save model

In [6]:
HelperUtils.save_model(Configuration.base_model_name, model,class_names)

Saving model base and it's 432 classes
INFO:tensorflow:Assets written to: ./models/base\assets


Test trained model

In [9]:
%%capture
from utils.testing_util import TestingUtil
results = TestingUtil.test_with_test_set(model, class_names)

Print test results

In [10]:
TestingUtil.pretty_print_results(results)

                                           image  class prediction Correct
0     c0_0_Con31_YGzI_original_1618300443534.jpg  10197      54383      No
1                      97E1_G5_1621423488237.jpg  10928      10928     Yes
2                      OX9N_G5_1621423533748.jpg  10928      10928     Yes
3       c2_3_P11_SkEJ_original_1619000719222.jpg  11153      11153     Yes
4                 P11_SINK_P11_1619000717769.jpg  11153      92950      No
..                                           ...    ...        ...     ...
306                 B5_Ef0H_B5_1619085989929.jpg  98100       3039      No
307  c10_13_Pin3_Faj3_original_1618574063580.jpg  98138      98138     Yes
308   c2_3_Axle2_7eYo_original_1618224172194.jpg  99008      99008     Yes
309     c0_2_i2L3_original-T30_1621412159729.jpg  99773      99773     Yes
310     c2_3_P51_IWno_original_1618573014049.jpg  99780      99780     Yes

[311 rows x 4 columns]
Test results: 277 / 311 (89.07%)
